In [1]:
import sys
# insert at 1, 0 is the script path (or '' in REPL)
sys.path.insert(1, './Preprocessing')
import numpy as np
import os
#os.environ["KERAS_BACKEND"] = "plaidml.keras.backend"
from imgDataGenerator import *
from preprocessing import *
import tensorflow as tf
import keras
from keras import applications
from keras.models import Sequential, Model, Input
from keras.layers import Conv2D, MaxPooling2D, LSTM, Reshape, Permute, TimeDistributed, CuDNNLSTM,LeakyReLU
from keras.layers import Activation, Dropout, Flatten, Dense, BatchNormalization
from keras.models import model_from_json

from keras.optimizers import SGD
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping
from keras.layers.merge import concatenate
#suppress warnings
import warnings
warnings.filterwarnings('ignore')

Using TensorFlow backend.


In [14]:
a=[0,0,1]
b=np.zeros((5,3))
b[:,:]=a
b

array([[0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.]])

In [ ]:
#data dimension will be: #bag * #frame * 244 * 244 * 3
#simple implementation of CNN>LSTM without multires


In [2]:
#Note for Test, Training, and Validation folders, they all need to have same categories
parentPath='/home/billy/Desktop/VideoClasBinned'
checkAndGeneratePath(parentPath)
testPath='/home/billy/Desktop/VideoClasBinned/Test'
trainPath='/home/billy/Desktop/VideoClasBinned/Training'
validPath='/home/billy/Desktop/VideoClasBinned/Validation'



Length Check on Folder Succeed


In [2]:
testPath='/home/billy/Desktop/VideoClasBinned/Test'
trainPath='/home/billy/Desktop/VideoClasBinned/Training'
validPath='/home/billy/Desktop/VideoClasBinned/Validation'

In [3]:
trainingGen=imgDataGenerator(trainPath)
validGen=imgDataGenerator(validPath)

In [4]:
combo=prepdata(trainPath)
len(combo)

395

In [5]:
a,b=zip(*combo)

In [6]:
len(a)

395

In [7]:
len(b)

395

In [8]:
a

('ADMINISTER MEDICATION',
 'ADMINISTER MEDICATION',
 'ADMINISTER MEDICATION',
 'ADMINISTER MEDICATION',
 'ADMINISTER MEDICATION',
 'ADMINISTER MEDICATION',
 'ADMINISTER MEDICATION',
 'ADMINISTER MEDICATION',
 'ADMINISTER MEDICATION',
 'ADMINISTER MEDICATION',
 'ADMINISTER MEDICATION',
 'ADMINISTER MEDICATION',
 'ADMINISTER MEDICATION',
 'ADMINISTER MEDICATION',
 'ADMINISTER MEDICATION',
 'ADMINISTER MEDICATION',
 'ADMINISTER MEDICATION',
 'ADMINISTER MEDICATION',
 'ADMINISTER MEDICATION',
 'BAGGING',
 'BAGGING',
 'BAGGING',
 'BAGGING',
 'BAGGING',
 'BAGGING',
 'BAGGING',
 'BLOOD-PRESSURE CUFF',
 'BLOOD-PRESSURE CUFF',
 'BLOOD-PRESSURE CUFF',
 'BLOOD-PRESSURE CUFF',
 'BLOOD-PRESSURE CUFF',
 'BLOOD-PRESSURE CUFF',
 'BLOOD-PRESSURE CUFF',
 'BLOOD-PRESSURE CUFF',
 'BLOOD-PRESSURE CUFF',
 'CHEST-TUBE',
 'CHEST-TUBE',
 'CHEST-TUBE',
 'CHEST-TUBE',
 'CHEST-TUBE',
 'CHEST-TUBE',
 'CHEST-TUBE',
 'CHEST-TUBE',
 'CHEST-TUBE',
 'CHEST-TUBE',
 'CHEST-TUBE',
 'CHEST-TUBE',
 'CHEST-TUBE PREP',
 'CHES

In [7]:
b=np.argmax(a,axis=1)
b=b.tolist()
b

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 12,
 12,
 12,
 12,
 12,
 

In [9]:
lval=0
mval=400
for i in range(23):
    tmp=b.count(i)
    if(tmp>lval):
        lval=tmp
    if(tmp<mval):
        mval=tmp
print('large: ', lval, 'mval: ', mval)

large:  54 mval:  5


In [4]:
#Using TimeDistributed Wrapper
inputData=Input(shape=(None, 244, 244, 3))
model = applications.inception_v3.InceptionV3(weights='imagenet', include_top=False,input_shape = ( 244, 244, 3))
#for layer in model.layers[:278]: #until mixed9_0, untrainable
#   layer.trainable = False
layer_name = 'mixed3'
intermediate_layer_model = Model(inputs=model.input,
                                 outputs=model.get_layer(layer_name).output)
for layer in intermediate_layer_model.layers: #until mixed9_0, untrainable
  layer.trainable = False
encoded_frame = TimeDistributed(intermediate_layer_model)(inputData)

encoded_frame = TimeDistributed(Flatten())(encoded_frame)
encoded_frame = TimeDistributed(Dense(64))(encoded_frame)
encoded_frame = BatchNormalization()(encoded_frame)
encoded_frame = TimeDistributed(LeakyReLU(alpha=0.1))(encoded_frame)
intermediateOut = CuDNNLSTM(128, return_sequences = True)(encoded_frame)
intermediateOut = BatchNormalization()(intermediateOut)
# intermediateOut = CuDNNLSTM(64)(intermediateOut)
# intermediateOut = BatchNormalization()(intermediateOut)

intermediateOut = TimeDistributed(Dense(32))(intermediateOut)
intermediateOut = TimeDistributed(BatchNormalization())(intermediateOut)
intermediateOut = TimeDistributed(LeakyReLU(alpha=0.1))(intermediateOut)
intermediateOut = TimeDistributed(Dropout(0.4))(intermediateOut)

# intermediateOut = Dense(32)(intermediateOut)
# intermediateOut = BatchNormalization()(intermediateOut)
# intermediateOut = LeakyReLU(alpha=0.1)(intermediateOut)

intermediateOut = TimeDistributed(Dropout(0.4))(intermediateOut)
prediction = TimeDistributed(Dense(29, activation='softmax'))(intermediateOut)
optmr = keras.optimizers.adam(lr=0.005, decay = 5e-4)
model_final = Model(input = [inputData], output = prediction)
model_final.compile(loss = "categorical_crossentropy", optimizer = optmr, metrics=["accuracy"])

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [5]:
model_final.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, None, 244, 244, 3) 0         
_________________________________________________________________
time_distributed_1 (TimeDist (None, None, 13, 13, 768) 2146976   
_________________________________________________________________
time_distributed_2 (TimeDist (None, None, 129792)      0         
_________________________________________________________________
time_distributed_3 (TimeDist (None, None, 64)          8306752   
_________________________________________________________________
batch_normalization_95 (Batc (None, None, 64)          256       
_________________________________________________________________
time_distributed_4 (TimeDist (None, None, 64)          0         
_________________________________________________________________
cu_dnnlstm_1 (CuDNNLSTM)     (None, None, 128)         99328     
__________

In [29]:
with tf.device('GPU:0'):
    model_final.fit_generator(generator=trainingGen,
                    validation_data=validGen,
                    use_multiprocessing=True,
                    epochs = 15,
                    workers=2)
    

Epoch 1/15
399/399 [==============================] - 136s 341ms/step - loss: 2.8297 - acc: 0.1941 - val_loss: 3.2185 - val_acc: 0.0000e+00
Epoch 2/15
399/399 [==============================] - 145s 364ms/step - loss: 2.7942 - acc: 0.2014 - val_loss: 3.2793 - val_acc: 3.6550e-04
Epoch 3/15
399/399 [==============================] - 147s 368ms/step - loss: 2.7749 - acc: 0.2043 - val_loss: 3.2404 - val_acc: 0.0000e+00
Epoch 4/15
399/399 [==============================] - 147s 369ms/step - loss: 2.7376 - acc: 0.2074 - val_loss: 3.2391 - val_acc: 0.0682
Epoch 5/15
228/399 [================>.............] - ETA: 55s - loss: 2.7189 - acc: 0.2083

Process ForkPoolWorker-44:
Process ForkPoolWorker-42:
Process ForkPoolWorker-43:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/billy/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/billy/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/billy/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/billy/anaconda3/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/home/billy/anaconda3/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/home/billy/anaconda3/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)


KeyboardInterrupt: 

  File "/home/billy/anaconda3/lib/python3.7/multiprocessing/pool.py", line 135, in worker
    completed += 1
  File "/home/billy/anaconda3/lib/python3.7/multiprocessing/pool.py", line 110, in worker
    task = get()
  File "/home/billy/anaconda3/lib/python3.7/multiprocessing/pool.py", line 110, in worker
    task = get()
KeyboardInterrupt
  File "/home/billy/anaconda3/lib/python3.7/multiprocessing/queues.py", line 352, in get
    res = self._reader.recv_bytes()
Process ForkPoolWorker-41:
  File "/home/billy/anaconda3/lib/python3.7/multiprocessing/connection.py", line 216, in recv_bytes
    buf = self._recv_bytes(maxlength)
  File "/home/billy/anaconda3/lib/python3.7/multiprocessing/connection.py", line 407, in _recv_bytes
    buf = self._recv(4)
  File "/home/billy/anaconda3/lib/python3.7/multiprocessing/queues.py", line 351, in get
    with self._rlock:
  File "/home/billy/anaconda3/lib/python3.7/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__

In [ ]:
#Using TimeDistributed Wrapper and freeze all CNN layer to mix8
inputData=Input(shape=(None, 244, 244, 3))
model = applications.inception_v3.InceptionV3(weights='imagenet', include_top=False,input_shape = ( 244, 244, 3))
for layer in model.layers: #fix all param and get output from mixed8
   layer.trainable = False
layer_name = 'mixed8'
intermediate_layer_model = Model(inputs=model.input,
                                 outputs=model.get_layer(layer_name).output)
for layer in intermediate_layer_model.layers: #fix all param and get output from mixed8
   layer.trainable = False
encoded_frame = TimeDistributed(intermediate_layer_model)(inputData)

encoded_frame=TimeDistributed(Flatten())(encoded_frame)
intermediateOut = CuDNNLSTM(128, return_sequences = True)(encoded_frame)
intermediateOut = BatchNormalization()(intermediateOut)
intermediateOut = CuDNNLSTM(64)(intermediateOut)
intermediateOut = BatchNormalization()(intermediateOut)

intermediateOut = Dense(64)(intermediateOut)
intermediateOut = BatchNormalization()(intermediateOut)
intermediateOut = LeakyReLU(alpha=0.1)(intermediateOut)
intermediateOut = Dropout(0.4)(intermediateOut)

intermediateOut = Dense(32)(intermediateOut)
intermediateOut = BatchNormalization()(intermediateOut)
intermediateOut = LeakyReLU(alpha=0.1)(intermediateOut)

intermediateOut = Dropout(0.4)(intermediateOut)
prediction = Dense(29, activation='softmax')(intermediateOut)
optmr_freeze = keras.optimizers.adam(lr=0.01, decay = 5e-4)
model_final_freeze = Model(input = [inputData], output = prediction)
model_final_freeze.compile(loss = "categorical_crossentropy", optimizer = optmr_freeze, metrics=["accuracy"])

In [ ]:
with tf.device('GPU:0'):
    model_final_freeze.fit_generator(generator=trainingGen,
                    validation_data=validGen,
                    use_multiprocessing=True,
                    epochs = 20,
                    workers=2)

In [ ]:
from tensorflow.python.client import device_lib

print(device_lib.list_local_devices())

In [ ]:
model_json = model_final.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model_final.save_weights("model.h5")

In [ ]:
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
model_final = model_from_json(loaded_model_json)
# load weights into new model
model_final.load_weights("model.h5")
print("Loaded model from disk")

In [31]:
#manually test model using testset
testGen=imgDataGenerator(testPath)
correct=0
laxcorrect=0
with tf.device('GPU:0'):
    for i in range(124):
        x,y=testGen.__getitem__(i)
        pred=model_final.predict(x)
        pred=pred.reshape((len(pred[0]),29))
        spred=np.sum(pred,axis=0)
        aspred=np.argsort(-spred)
#         print(aspred[:3])
#         print(aspred)
#         print(np.argmax(spred))
#         print(spred)
#         print(np.argmax(y[0][0]))
#         print('**')
        if(np.argmax(y[0][0]) in aspred[:2]):
            correct+=1
        if(np.argmax(y[0][0]) in aspred[:4]):
            laxcorrect+=1    
print('Accuracy of Testing Set: ',correct/124, laxcorrect/124)

Accuracy of Testing Set:  0.23387096774193547 0.4838709677419355


In [6]:
a=[1,2,3]
a=np.array(a)
b=[4,5,6]
b=np.array(b)
a+b

array([5, 7, 9])

In [13]:
c=np.array([[1,2,3],[3,4,5]])
np.sum(c,axis=0)

array([4, 6, 8])

In [8]:
len(c)

2